In [159]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException, TimeoutException

from bs4 import BeautifulSoup
import io
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
import time
import sys
import pandas as pd
import re

In [176]:
class drugscom:
    def __init__(self):
        self.wurl = 'https://www.drugs.com/pill_identification.html'
        self.base = "https://www.drugs.com"
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--no-sandbox')
        self.driver = webdriver.chrome.webdriver.WebDriver('/usr/local/bin/chromedriver', options=chrome_options)
        self.wait = WebDriverWait(self.driver, 10)
        self.driver.set_window_size(850,1600)
#         self.driver.get(self.wurl)
#         WebDriverWait(self.driver, 100).until(EC.title_contains("Pill Identifier (Pill Finder) - Drugs.com"))
#         print('page loaded')
#         time.sleep(5)
#         elem = self.wait.until(EC.element_to_be_clickable((By.LINK_TEXT, 'Accept')))
#         elem = self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR , "a .ddc-btn")))
#         elem = self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR , "a .ddc-btn")))
#         print('clickable')
#         elem.click()
        self.results = []
        
    def get_data(self, pmprint): 
        self.driver.get(self.wurl)
        WebDriverWait(self.driver, 100).until(EC.title_contains("Pill Identifier (Pill Finder) - Drugs.com"))  
        elem = self.wait.until(EC.element_to_be_clickable((By.LINK_TEXT, 'Accept')))
        elem.click()
        mprint = pmprint
#         elem = self.wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@href='/imprints.php']")))
        elem = self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR , "#livesearch-imprint")))
        elem.click()
        elem.clear()
        elem.send_keys(mprint)
        elem.send_keys(Keys.RETURN)
        elem = self.wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@type='submit']")))
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        for elem in soup(text='Pill Imprint:'):
            a = elem.parent.find_next('a')
#             print(f"img: {a['href']} MPRINT: {a.text}")
            mprint = a.text
            self.driver.get(base + a['href'])
            WebDriverWait(self.driver, 100).until(EC.title_contains(mprint))
#             print(a.text, ' title')
            isoup = BeautifulSoup(self.driver.page_source, 'html.parser')
            imgs = isoup.find_all('img')
#             print('imgs len',len(imgs))
            for img in imgs:
#                   print('img', img)
                  try:
                      s = img['src']
#                       print('s',s)
                      if s[0:18] == '/images/pills/nlm/':
#                          print('found img', s, ' mprint ', mprint)
#                         self.img = base + s
                         self.results.append({'mprint': mprint, 'img': self.base + s})
                         break
                  except:
                      pass
            if mprint == pmprint:
                break
        return self.results
    
    def reset(self):
        self.results = []


In [177]:
d = drugscom()
d.get_data("C 10")
d.get_data("LILLY 3240 30 mg")
d.get_data("LILLY 4117")

[{'mprint': 'C 10',
  'img': 'https://www.drugs.com/images/pills/nlm/200000010.jpg'},
 {'mprint': 'Lilly 3240 30 mg',
  'img': 'https://www.drugs.com/images/pills/nlm/000023240.jpg'},
 {'mprint': 'LILLY 4117',
  'img': 'https://www.drugs.com/images/pills/nlm/000024117.jpg'}]

In [178]:
d.get_data("10 C")

[{'mprint': 'C 10',
  'img': 'https://www.drugs.com/images/pills/nlm/200000010.jpg'},
 {'mprint': 'Lilly 3240 30 mg',
  'img': 'https://www.drugs.com/images/pills/nlm/000023240.jpg'},
 {'mprint': 'LILLY 4117',
  'img': 'https://www.drugs.com/images/pills/nlm/000024117.jpg'},
 {'mprint': 'PERCOCET 10/325',
  'img': 'https://www.drugs.com/images/pills/nlm/634810629.jpg'},
 {'mprint': 'BAC 10 832',
  'img': 'https://www.drugs.com/images/pills/nlm/008321024.jpg'},
 {'mprint': 'M EC 10',
  'img': 'https://www.drugs.com/images/pills/nlm/200000822.jpg'}]

In [179]:
d.reset()
d.get_data("4117 LILLY")

[{'mprint': 'LILLY 4117',
  'img': 'https://www.drugs.com/images/pills/nlm/000024117.jpg'}]